# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [4]:
import os
import openai
from openai import OpenAI

In [5]:
llm_model = "gpt-4"

In [11]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

### ConversationBufferMemory

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [12]:
llm = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=os.environ['OPENAI_API_KEY'])

In [13]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True)

In [16]:
conversation.predict(input='Hi, my name is andrew')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is andrew
AI:

> Finished chain.


"Hello, Andrew! It's nice to meet you. How can I assist you today?"

In [17]:
conversation.predict(input='What is 1+1')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andrew
AI: Hello, Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI:

> Finished chain.


'The sum of 1 + 1 is 2.'

In [18]:
conversation.predict(input='what is my name?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andrew
AI: Hello, Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI: The sum of 1 + 1 is 2.
Human: what is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [19]:
print(memory.buffer)

Human: Hi, my name is andrew
AI: Hello, Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI: The sum of 1 + 1 is 2.
Human: what is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is andrew\nAI: Hello, Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1\nAI: The sum of 1 + 1 is 2.\nHuman: what is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [22]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [23]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is andrew\nAI: Hello, Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1\nAI: The sum of 1 + 1 is 2.\nHuman: what is my name?\nAI: Your name is Andrew, as you mentioned earlier.\nHuman: Not much, just hanging\nAI: Cool"}

#### ConversationBufferWindowMemory

In [24]:
from langchain.memory import ConversationBufferWindowMemory

In [25]:
memory = ConversationBufferWindowMemory(k=1)

In [26]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [27]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [28]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [29]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [30]:
conversation.predict(input="What is 1+1?")

'The sum of 1+1 is 2.'

In [31]:
conversation.predict(input="What is my name?")

"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. So, I don't know your name."

#### ConversationTokenBufferMemory

In [32]:
from langchain.memory import ConversationTokenBufferMemory

In [38]:
memory = ConversationTokenBufferMemory(
    llm=llm,
    model=llm_model)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [39]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [40]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [42]:
conversation.predict(input='')

'Delightful!'

#### ConversationSummaryMemory

In [49]:
from langchain.memory import ConversationSummaryBufferMemory

In [50]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [52]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and the human asks about their schedule for the day. The AI informs them of a morning meeting with the product team, time set aside for working on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in the latest in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo.'}

In [53]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [54]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and the human asks about their schedule for the day. The AI informs them of a morning meeting with the product team, time set aside for working on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in the latest in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be the latest version of our Language Learning Model (LLM). This model has been updated with new features such as real-time translation, personalized language learning paths, and an interactive interface. It would be beneficial to demonstrate how these features work in real-time. Additionally, you could show the customer how the model adapts to different learning styles and paces, which could be a key selling point.'

In [55]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and the human asks about their schedule for the day. The AI informs them of a morning meeting with the product team, time set aside for working on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in the latest in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo. The human then asks the AI for suggestions on a good demo to show.\nAI: A good demo to show would be the latest version of our Language Learning Model (LLM). This model has been updated with new features such as real-time translation, personalized language learning paths, and an interactive interface. It would be beneficial to demonstrate how these features work in real-time. Additionally, you could show the customer how the model adapts to different learning styles and paces, which could be a key selling point.'}